In [1]:
using PyCall

using DrWatson
quickactivate(@__DIR__)

include(projectdir("scripts", "notebooks_netsense_funs.jl"))

using CairoMakie

In [2]:
fname = datadir("exp_pro", "NetSense_network.pkl")
file = load_pickle(fname)

Dict{Any, Any} with 6 entries:
  5 => PyObject <networkx.classes.graph.Graph object at 0x7faf3487d580>
  4 => PyObject <networkx.classes.graph.Graph object at 0x7faf348c2940>
  6 => PyObject <networkx.classes.graph.Graph object at 0x7faf3483f880>
  2 => PyObject <networkx.classes.graph.Graph object at 0x7faf349600a0>
  3 => PyObject <networkx.classes.graph.Graph object at 0x7faf34905ca0>
  1 => PyObject <networkx.classes.graph.Graph object at 0x7faf633a73a0>

In [3]:
As = [get_adjacency_matrix(file[i]) for i in 1:6]
A1 = get_adjacency_matrix(file[1])
A2 = As[2]
A1

98×98 Matrix{Int64}:
  0  11    0    4   2   0   0   0   0  …  0  0  0   0   0  0   0  0  0  0  0
 11   0   61    8   0   0   0   0   0     0  0  0   0   0  0   0  0  0  0  0
  0  61    0  193  19   0   0   0   0     0  0  0   0   0  0   0  0  0  0  0
  4   8  193    0   5   0   0   0   8     0  0  0   0   0  0   0  0  0  0  0
  2   0   19    5   0   0   0   7   0     0  0  0   0   0  0   0  1  0  0  0
  0   0    0    0   0   0  18  36  49  …  0  0  0   0   0  0   0  0  0  0  0
  0   0    0    0   0  18   0   0   0     0  0  0   0   0  0   0  2  0  0  0
  0   0    0    0   7  36   0   0  81     0  0  0   0   0  0   0  0  0  0  0
  0   0    0    8   0  49   0  81   0     0  0  0   0   0  0   0  0  0  0  0
  0   0    0    0   0   0   0   0   0     0  0  0   0   0  0   0  0  0  0  0
  ⋮                     ⋮              ⋱            ⋮                ⋮     
  0   0    0    0   0   0   0   0   0     0  0  0   0   0  0   0  0  0  0  0
  0   0    0    0   0   0   0   0   0  …  0  0  0   0  8

In [4]:
nodes = get_all_nodes(file)
Ws = create_series_adj_mat(file, nodes)
As = [sign.(W) for W in Ws]

all_triads = [get_triads2(As[i]) for i in 1:6];

In [5]:
# Let's check how many people are in triads

triad_nodes = sort(unique([node for triads in all_triads for triad in triads for node in triad]))
display(length(triad_nodes))

# There are only 60 nodes, so lets repeat above experiment with generating links only among these agents. 
# But first I need to count how many edges were added among those agents

As2 = [As[i][triad_nodes, triad_nodes] for i in 1:6]

60

6-element Vector{Matrix{Int64}}:
 [0 1 … 0 0; 1 0 … 0 0; … ; 0 0 … 0 0; 0 0 … 0 0]
 [0 0 … 0 0; 0 0 … 0 0; … ; 0 0 … 0 0; 0 0 … 0 0]
 [0 0 … 0 0; 0 0 … 0 0; … ; 0 0 … 0 0; 0 0 … 0 0]
 [0 0 … 0 0; 0 0 … 0 1; … ; 0 0 … 0 0; 0 1 … 0 0]
 [0 0 … 0 0; 0 0 … 0 0; … ; 0 0 … 0 0; 0 0 … 0 0]
 [0 0 … 0 0; 0 0 … 0 0; … ; 0 0 … 0 0; 0 0 … 0 0]

In [6]:
#creating list of attributes
list_of_attributes = Set([k for n in file[1].nodes for k in keys(file[1].nodes[n+1])])

# creating possible attribute values
attribute_values = Dict((attribute, Set() ) for attribute in list_of_attributes)

for sem in 1:6
    for node in file[sem].nodes()
        for attribute in list_of_attributes
            push!(attribute_values[attribute], convert(Int, file[sem].nodes[node + 1][attribute]))
        end
    end
end

attribute_values

#To apply PCA or other analysis on all attributes (i.e. including ethnicity) 
#one would have to create an attribute for each race, that is is_1, is_2 etc. 
#Because this attribute is not ordered. 

Dict{String, Set{Any}} with 12 entries:
  "premaritalsex_" => Set([2, 3, 1])
  "ethnicity_1"    => Set([4, 2, 3, 1])
  "hometown_1"     => Set([2, 3, 1])
  "political_"     => Set([2, 3, 1])
  "abortion_"      => Set([2, 3, 1])
  "homosexual_"    => Set([2, 3, 1])
  "marijuana_"     => Set([2, 3, 1])
  "gender_1"       => Set([2, 1])
  "gaymarriage_"   => Set([2, 3, 1])
  "deathpen_"      => Set([2, 3, 1])
  "euthanasia_"    => Set([2, 3, 1])
  "age_1"          => Set([2, 1])

In [7]:
#creating table of attribute values
attributes = Dict((i, zeros(length(nodes), length(list_of_attributes))) for i in 1:6)
# for i in 1:6
#     attributes[i] = zeros(length(file[i].nodes), length(list_of_attributes))
# end

for sem in 1:6
    for (i, node) in enumerate(nodes)
        for (j, attribute) in enumerate(list_of_attributes)
            if node in file[sem].nodes()
                attributes[sem][i,j] = file[sem].nodes[node + 1][attribute]
            else
                attributes[sem][i,j] = NaN
            end
        end
    end
end
attributes

Dict{Int64, Matrix{Float64}} with 6 entries:
  5 => [3.0 3.0 … 2.0 1.0; 3.0 3.0 … 2.0 2.0; … ; NaN NaN … NaN NaN; 1.0 3.0 … …
  4 => [3.0 3.0 … 2.0 1.0; 3.0 3.0 … 2.0 1.0; … ; 1.0 1.0 … 1.0 1.0; 1.0 3.0 … …
  6 => [NaN NaN … NaN NaN; 3.0 3.0 … 2.0 2.0; … ; NaN NaN … NaN NaN; NaN NaN … …
  2 => [NaN NaN … NaN NaN; 3.0 3.0 … 2.0 2.0; … ; NaN NaN … NaN NaN; NaN NaN … …
  3 => [3.0 3.0 … 2.0 1.0; 3.0 1.0 … 2.0 2.0; … ; NaN NaN … NaN NaN; NaN NaN … …
  1 => [3.0 3.0 … 2.0 1.0; 2.0 2.0 … 2.0 1.0; … ; NaN NaN … NaN NaN; NaN NaN … …

In [8]:

ord_list_of_attributes = setdiff(list_of_attributes, ["ethnicity_1", "gender_1", "hometown_1", "age_1"])
print(ord_list_of_attributes)

#creating table of attribute values
ord_attributes = Dict((i, zeros(length(nodes), length(ord_list_of_attributes))) for i in 1:6)

for sem in 1:6
    for (i, node) in enumerate(nodes)
        for (j, attribute) in enumerate(ord_list_of_attributes)
            if node in file[sem].nodes()
                ord_attributes[sem][i,j] = file[sem].nodes[node + 1][attribute]
            else
                ord_attributes[sem][i,j] = NaN
            end
        end
    end
end
# for sem in range(1,7):
#     for i, node in enumerate(data[sem].nodes()):
#         for j, attribute in enumerate(ord_list_of_attributes):
#             ord_attributes[sem][i,j] = data[sem].nodes[node][attribute]
ord_attributes

Set(["abortion_", "premaritalsex_", "homosexual_", "marijuana_", "euthanasia_", "gaymarriage_", "deathpen_", "political_"])

Dict{Int64, Matrix{Float64}} with 6 entries:
  5 => [3.0 3.0 … 3.0 1.0; 3.0 3.0 … 3.0 2.0; … ; NaN NaN … NaN NaN; 1.0 3.0 … …
  4 => [3.0 3.0 … 3.0 1.0; 3.0 3.0 … 1.0 1.0; … ; 1.0 1.0 … 3.0 1.0; 1.0 3.0 … …
  6 => [NaN NaN … NaN NaN; 3.0 3.0 … 3.0 2.0; … ; NaN NaN … NaN NaN; NaN NaN … …
  2 => [NaN NaN … NaN NaN; 3.0 3.0 … 1.0 2.0; … ; NaN NaN … NaN NaN; NaN NaN … …
  3 => [3.0 3.0 … 3.0 1.0; 3.0 1.0 … 1.0 2.0; … ; NaN NaN … NaN NaN; NaN NaN … …
  1 => [3.0 3.0 … 3.0 1.0; 2.0 2.0 … 2.0 1.0; … ; NaN NaN … NaN NaN; NaN NaN … …

## Clustering coefficient and other simple statistics

In [11]:
nets = [Graph(a) for a in As]

6-element Vector{SimpleGraph{Int64}}:
 {103, 152} undirected simple Int64 graph
 {103, 116} undirected simple Int64 graph
 {103, 61} undirected simple Int64 graph
 {103, 109} undirected simple Int64 graph
 {103, 101} undirected simple Int64 graph
 {103, 69} undirected simple Int64 graph

In [19]:
#Global clustering coefficient (transitivity)
gcc = [global_clustering_coefficient(nets[i]) for i in 1:6]

6-element Vector{Float64}:
 0.22203947368421054
 0.2591362126245847
 0.3333333333333333
 0.2746478873239437
 0.24193548387096775
 0.2482758620689655

In [38]:
#average clustering coefficient
local_clustering(nets[1])

([2, 2, 2, 5, 3, 13, 0, 14, 13, 2  …  0, 1, 0, 0, 0, 0, 0, 0, 0, 0], [3, 6, 6, 36, 15, 36, 10, 78, 55, 10  …  6, 3, 0, 0, 0, 0, 0, 0, 0, 0])

In [40]:
#average clustering coefficient
acc = [mean(local_clustering_coefficient(nets[i])) for i in 1:6]

6-element Vector{Float64}:
 0.1828862076434892
 0.20957004160887654
 0.14757281553398058
 0.2502542764678687
 0.17722299275697334
 0.14373555247341654

In [29]:
#Active agents
Ns = [sum(sum(As[i], dims=1) .> 0) for i in 1:6]

6-element Vector{Int64}:
 98
 89
 68
 89
 85
 63

In [41]:
#average clustering coefficient including only active agents
acc = [sum(local_clustering_coefficient(nets[i])) / Ns[i] for i in 1:6]

6-element Vector{Float64}:
 0.1922171366048917
 0.24253611556982344
 0.22352941176470587
 0.28962011771000534
 0.21475256769374415
 0.2349962207105064

In [42]:
#average clustering coefficient including all 108 agents
acc = [sum(local_clustering_coefficient(nets[i])) / 108 for i in 1:6]

6-element Vector{Float64}:
 0.17441925358592025
 0.19986772486772486
 0.14074074074074072
 0.238668430335097
 0.16901822457378013
 0.1370811287477954

In [33]:
# Graph density, assuming active agents only
gd = [sum(As[i])/2/(Ns[i] * (Ns[1]-1)/2) for i in 1:6]

6-element Vector{Float64}:
 0.03197980223017042
 0.026873624464265028
 0.018496058217101274
 0.025251940229352485
 0.024499696785930867
 0.022582228767795778

In [36]:
gd .^ 3 .* Ns .* (Ns .-1) .* (Ns .-2) ./ 6

6-element Vector{Float64}:
 4.974450499311474
 2.204039700212957
 0.3171129309562155
 1.8286262050303712
 1.4524700372396004
 0.4573105616881114